## Laboratory 2
### Aaron Sigman

There are more satellites and data sources available to you. Using your Lab 2 notebook and the one we showed how to create your own notebook (attached) create a notebook adding the following satellites in natural and read false color display (both).

In [1]:
from IPython.display import Image
import ee
ee.Initialize()

In [2]:
import geemap

### Define a point

In [3]:
# Define point at Logan, UT
Lat=41.735210
Long=-111.834862
Point = ee.Geometry.Point([Long, Lat])

In [4]:
landsat8 =ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") #change
image = ee.Image(landsat8
    .filterBounds(Point)
    .filterDate('2021-06-01', '2021-09-01')
    .sort('CLOUD_COVER_LAND')
    .first())
image = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])
image = image.multiply(0.0000275).add(-0.2)

In [5]:
Map = geemap.Map() # from ipygee
bounds = Point.buffer(10000)
Map.centerObject(bounds)
trueColor = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.4}
falseColor= {'bands': ['SR_B5', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.4}
Map.addLayer(image, trueColor, name='Landsat8 True')
Map.addLayer(image, falseColor, name='Landsat8 False')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
ASTER = ee.ImageCollection("ASTER/AST_L1T_003")
# // Get ASTER images for the study period and region of interest.
ASTERImages = ASTER.filterDate('2019-04-01', '2019-10-31').filterBounds(Point);
# // Mosaic adjacent images into a single image.
ASTERImage = ASTERImages.mosaic();
# Print the image band names to the console.
band_names = ASTERImage.bandNames()
print(band_names.getInfo())

['B01', 'B02', 'B3N', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14']


In [7]:
IRBands=  ['B3N', 'B02', 'B01']

Map.addLayer(ASTERImage, {'bands':IRBands,'min': 0, 'max': 255}, name='ASTER Image')
Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

In [8]:
SEN2 = ee.ImageCollection("COPERNICUS/S2")

# // Get ASTER images for the study period and region of interest.
SEN2Images = SEN2.filterDate('2019-04-01', '2019-10-31').filterBounds(Point);

# // Mosaic adjacent images into a single image.
SEN2Image = SEN2Images.mosaic();
SEN2Image = SEN2Image.multiply(0.0001)
# Print the image band names to the console.
band_names = SEN2Image.bandNames()
print(band_names.getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


In [9]:
# // Display the SEN2 mosaic as a color-IR composite.
IRBands=  ['B8', 'B4', 'B3']
TrueBands=['B4','B3','B2']
Map.addLayer(SEN2Image, {'bands':IRBands,'min': 0, 'max': 1}, name='SEN2 NIR')
Map.addLayer(SEN2Image, {'bands':TrueBands,'min': 0, 'max': 1}, name='SEN2 True')
Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
SEN3 = ee.ImageCollection("COPERNICUS/S3/OLCI").filterDate('2019-04-01', '2019-10-31').filterBounds(Point)
rgb = SEN3.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance']).median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
NIR = SEN3.select(['Oa14_radiance','Oa08_radiance', 'Oa06_radiance']).median().multiply(ee.Image([0.0086512, 0.00876539, 0.0123538]))

visParams = {'min': 0,'max': 4,'gamma': 1.5}

In [11]:
Map.addLayer(rgb, visParams, name='SEN3 True')
Map.addLayer(NIR, visParams, name='SEN3 NIR')

Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
PROBA = ee.ImageCollection("VITO/PROBAV/C1/S1_TOC_100M").filterDate('2019-05-25', '2019-10-27').filterBounds(Point).sort("CLOUD_COVER_LAND").first()
band_names = PROBA.bandNames()
print(band_names.getInfo())

['RED', 'NIR', 'BLUE', 'SWIR', 'NDVI', 'SZA', 'SAA', 'SWIRVAA', 'SWIRVZA', 'VNIRVAA', 'VNIRVZA', 'SM', 'TIME']


In [13]:
# No green band, so combo here is NIR, R, B
IRBands=  ['NIR', 'RED', 'BLUE']
Map.addLayer(PROBA, {'bands':IRBands,'min': 20, 'max': 2000}, name='PROBA IR')
Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
VIIRS = ee.ImageCollection("NOAA/VIIRS/001/VNP09GA")

# // Get ASTER images for the study period and region of interest.
VIIRSImages = VIIRS.filterDate('2019-05-25', '2019-10-27').filterBounds(Point).sort("CLOUD_COVER_LAND").first()

VIIRSImage = VIIRSImages.multiply(0.0001)
# Print the image band names to the console.
band_names = VIIRSImage.bandNames()
print(band_names.getInfo())

['I1', 'I2', 'I3', 'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8', 'M10', 'M11', 'SensorAzimuth', 'SensorZenith', 'SolarAzimuth', 'SolarZenith', 'iobs_res', 'num_observations_1km', 'num_observations_500m', 'obscov_1km', 'obscov_500m', 'orbit_pnt', 'QF1', 'QF2', 'QF3', 'QF4', 'QF5', 'QF6', 'QF7']


In [15]:
IRBands=  ['M7', 'M5', 'M4']
TrueBands=['M5','M4','M3']
Map.addLayer(VIIRSImage, {'bands':IRBands,'min': 0, 'max': 1}, name='VIIRS IR')
Map.addLayer(VIIRSImage, {'bands':TrueBands,'min': 0, 'max': 1}, name='VIIRS True')

Map

Map(center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['position', 'transparent_…

In [16]:
HYP = ee.ImageCollection("EO1/HYPERION")

# // Get ASTER images for the study period and region of interest.
HYPImages = HYP.filterDate('2001-05-01', '2002-5-01').filterBounds(Point);

# // Mosaic adjacent images into a single image.
HYPImage = HYPImages.mosaic();
#VIIRSImage = VIIRSImage.multiply(0.0001)
# Print the image band names to the console.
band_names = HYPImage.bandNames()
print(band_names.getInfo())

['B008', 'B009', 'B010', 'B011', 'B012', 'B013', 'B014', 'B015', 'B016', 'B017', 'B018', 'B019', 'B020', 'B021', 'B022', 'B023', 'B024', 'B025', 'B026', 'B027', 'B028', 'B029', 'B030', 'B031', 'B032', 'B033', 'B034', 'B035', 'B036', 'B037', 'B038', 'B039', 'B040', 'B041', 'B042', 'B043', 'B044', 'B045', 'B046', 'B047', 'B048', 'B049', 'B050', 'B051', 'B052', 'B053', 'B054', 'B055', 'B056', 'B057', 'B077', 'B078', 'B079', 'B080', 'B081', 'B082', 'B083', 'B084', 'B085', 'B086', 'B087', 'B088', 'B089', 'B090', 'B091', 'B092', 'B093', 'B094', 'B095', 'B096', 'B097', 'B098', 'B099', 'B100', 'B101', 'B102', 'B103', 'B104', 'B105', 'B106', 'B107', 'B108', 'B109', 'B110', 'B111', 'B112', 'B113', 'B114', 'B115', 'B116', 'B117', 'B118', 'B119', 'B120', 'B121', 'B122', 'B123', 'B124', 'B125', 'B126', 'B127', 'B128', 'B129', 'B130', 'B131', 'B132', 'B133', 'B134', 'B135', 'B136', 'B137', 'B138', 'B139', 'B140', 'B141', 'B142', 'B143', 'B144', 'B145', 'B146', 'B147', 'B148', 'B149', 'B150', 'B151',

In [21]:
# NIR, R, G
IRBands=  ['B050', 'B023', 'B015']
TrueBands=['B023','B015','B013']
Map.addLayer(HYPImage, {'bands':IRBands,'min': 0, 'max': 7000}, name='HYPERION IR')
Map.addLayer(HYPImage, {'bands':TrueBands,'min': 0, 'max': 7000}, name='HYPERION True')
Map

Map(bottom=780850.0, center=[41.73775991204252, -111.81833267211915], controls=(WidgetControl(options=['positi…